# Overview
The issue of missing packages has plagued the postal and shipping industry for as long as one can remember. This is especially costly (in terms of both money and reputation) in the modern day and age, where e commerce companies regularly ship out valuable goods to their paying customers.

Through the course of this notebook, I intend to showcase the potential for using logistic regression to save costs in an e commerce company by building a model to identify the parcels to apply for insurance for. 

In this Hypothetical example, we managed to save **$457253.78** in insurance costs for 22,581 parcels.

### Note:
1. Insurance rates have been defined as **insurance_cost = max($20, 10% the cost of the order)** for this example.

2. We are assuming that a random 5% of all parcels get stolen. Hence, we **calibrate the model** to account for this unbalance.

3. Total reveune generated from the 22,581 parcels stands at **$6531261.43**

**By : [Naimish Mani B](https://www.linkedin.com/in/naimish-balaji-a6182b180/)**

In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-purchase-history-from-jewelry-store/jewelry.csv


# Introducton

There are 3 cases wherein the customer can claim to have not recieved a package they had paid for: 

1. Package was stolen from the customer before they could recieve it.
2. Package was lost in transit.
3. The customer is lying to get a refund.

Regardless of the case, usually the company offers a refund to the customer. But at the end of the day, it amounts to revenue lost from the company's side. To minimise the losses from the company's end, the e commerce can opt to insure the packages that are being shipped out.

But again, insurance usually comes at a fixed cost. Hence, for young startups that have just started out and are struggling to maintain good cashflow, insuring all packages might not be feasible. Hence, they might want to selectively insure only certain packages.

---

# The Dataset

First, let's load the datset into memory, and list out the columns. Next, we inspect it by calling `df.head()`

In [176]:
df = pd.read_csv('/kaggle/input/ecommerce-purchase-history-from-jewelry-store/jewelry.csv')
print(df.columns)
df.head()

Index(['event_time', 'order_id', 'product_id', 'quantity', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'gender', 'color',
       'metal', 'gem'],
      dtype='object')


,event_time,order_id,product_id,quantity,category_id,category_code,brand,price,user_id,gender,color,metal,gem
0,2018-11-29 16:30:45 UTC,1923415742179443254,1836250225916772582,1,1.806829e+18,jewelry.pendant,0.0,67.78,1.515916e+18,NaN,red,gold,diamond
1,2018-11-29 16:52:07 UTC,1923426489303302817,1836015460420681761,1,1.806829e+18,jewelry.pendant,0.0,32.63,1.515916e+18,NaN,red,gold,NaN
2,2018-11-29 17:58:37 UTC,1923459963229831173,1806829194936582544,1,1.806829e+18,jewelry.ring,1.0,75.21,1.515916e+18,NaN,red,gold,amethyst
3,2018-11-29 20:25:52 UTC,1923534078074684181,1835566854668550661,1,1.806829e+18,jewelry.earring,0.0,131.37,1.515916e+18,f,red,gold,NaN
4,2018-11-29 20:30:01 UTC,1923536169069445939,1836568752905257618,1,1.806829e+18,jewelry.bracelet,0.0,102.60,1.515916e+18,f,red,gold,NaN


In [199]:
# Calculate total revenue
df['price'].sum()

32997044.47

For the task at hand, we'll only be requiring the following columns:
   - event_time
   - order_id
   - user_id
   - price

Hence, we drop the other columns directly.

In [177]:
dataset = df.drop(columns=['product_id', 'quantity', 'category_id', 'category_code', 'brand', 'gender', 'color', 'metal', 'gem'])
dataset.head()

,event_time,order_id,price,user_id
0,2018-11-29 16:30:45 UTC,1923415742179443254,67.78,1.515916e+18
1,2018-11-29 16:52:07 UTC,1923426489303302817,32.63,1.515916e+18
2,2018-11-29 17:58:37 UTC,1923459963229831173,75.21,1.515916e+18
3,2018-11-29 20:25:52 UTC,1923534078074684181,131.37,1.515916e+18
4,2018-11-29 20:30:01 UTC,1923536169069445939,102.60,1.515916e+18


In [178]:
print("Number of elements: ", len(dataset))
print("Number of unique orders: ", len(dataset['order_id'].unique()))

Number of elements:  113302
Number of unique orders:  85604


As we can see, the number of unique orders < total number of rows in the dataset. This is because there are instances of some orders having multiple products in them.

**For simplicity sake, we will be assuming that each order gets shipped in a seperate box, and hence each box will have to be insured seperately.**

In [179]:
dataset.describe()

,order_id,price,user_id
count,1.133020e+05,112905.000000,1.129050e+05
mean,2.200117e+18,292.254944,1.512850e+18
std,1.410954e+17,481.857658,2.250084e+16
min,1.923416e+18,0.920000,1.313554e+18
25%,2.086874e+18,110.580000,1.515916e+18
50%,2.201607e+18,204.380000,1.515916e+18
75%,2.317472e+18,346.530000,1.515916e+18
max,2.453171e+18,34723.950000,1.555252e+18


Next, we convert all timestamps from the current format to Unix time, to make comparisions easier down the line.

In [180]:
dataset['event_time'] = dataset['event_time'].str.slice(0, -4)
dataset['event_time'] = (pd.DatetimeIndex(dataset['event_time']).astype(np.int64) // 10**9) * 1000
dataset.head()

,event_time,order_id,price,user_id
0,1543509045000,1923415742179443254,67.78,1.515916e+18
1,1543510327000,1923426489303302817,32.63,1.515916e+18
2,1543514317000,1923459963229831173,75.21,1.515916e+18
3,1543523152000,1923534078074684181,131.37,1.515916e+18
4,1543523401000,1923536169069445939,102.60,1.515916e+18


Now, we calculate the number of previous times the customer has made purchases on the site. We will be using this value as an input to the model later.

In [181]:
def f(x, y):
    foo = dataset[(dataset['user_id'] == y) & (dataset['event_time'] < x)]
    return len(foo)
    
prev_orders = [f(x, y) for x, y in zip(dataset['event_time'], dataset['user_id'])]
prev_orders[-100:]

[0,
 0,
 218,
 2,
 2,
 259,
 259,
 0,
 0,
 0,
 330,
 51,
 1,
 0,
 0,
 1,
 19,
 0,
 0,
 59,
 0,
 0,
 10,
 154,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 261,
 261,
 0,
 263,
 263,
 0,
 120,
 1,
 0,
 293,
 0,
 344,
 1,
 25,
 25,
 3,
 0,
 0,
 21,
 21,
 11,
 7,
 1,
 0,
 4,
 4,
 0,
 23,
 23,
 0,
 0,
 2,
 2,
 2,
 32,
 12,
 331,
 7,
 5,
 49,
 1,
 5,
 1,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 1,
 0,
 20,
 0,
 2,
 116,
 116,
 0,
 68,
 167,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0]

Next, we add this column to the dataset and get rid of the 'order_id' column, as we don't need it anymore.

In [182]:
# Add new column to dataset
dataset['prev_orders'] = prev_orders
# Delete old column from dataset
dataset = dataset.drop(columns=['order_id'])
dataset.head()

,event_time,price,user_id,prev_orders
0,1543509045000,67.78,1.515916e+18,0
1,1543510327000,32.63,1.515916e+18,0
2,1543514317000,75.21,1.515916e+18,0
3,1543523152000,131.37,1.515916e+18,0
4,1543523401000,102.60,1.515916e+18,0


Now unfortunately for us, the dataset does not contain data as to whether the customer recieved their order successfully or not. Hence, we generate fictional data for this using the following assumptions:

    - Each package has a 5% chance of getting stolen
    - This probability is random, and not dependent on any other parameters.

These assumptions are satisfied by using the `random` module in python.

In [183]:
from random import random
l = [0] * len(dataset['price'])
for i in range(len(l)):
    if random() < 0.05:
        l[i] = 1

dataset['lost'] = l
print(dataset['lost'].value_counts())
dataset.head()

0    107630
1      5672
Name: lost, dtype: int64


,event_time,price,user_id,prev_orders,lost
0,1543509045000,67.78,1.515916e+18,0,0
1,1543510327000,32.63,1.515916e+18,0,0
2,1543514317000,75.21,1.515916e+18,0,0
3,1543523152000,131.37,1.515916e+18,0,0
4,1543523401000,102.60,1.515916e+18,0,0


Next, we need to calculate the insurance cost for each order. That is calculated according to the following definition:

**insurance_cost = max($20, 10% the cost of the order)**

This is implemented below.

In [184]:
def g(x):
    return max(20, x*0.1)
    
insurance = [g(x) for x in dataset['price']]
insurance[-10:]

[20, 63.49, 63.49, 21.573, 41.703, 20, 20.948, 20, 23.959000000000003, 20]

In [185]:
dataset['insurance'] = insurance
dataset = dataset.drop(columns=['event_time', 'user_id'])
dataset.head()

,price,prev_orders,lost,insurance
0,67.78,0,0,20.0
1,32.63,0,0,20.0
2,75.21,0,0,20.0
3,131.37,0,0,20.0
4,102.60,0,0,20.0


Now that we have the whole dataset ready, we split it up into train and test sets using sklearn.

In [186]:
from sklearn.model_selection import train_test_split

dataset = dataset.dropna()
train, test = train_test_split(dataset, test_size=0.2)
print(train.head())
print(len(train))
print(test.head())
print(len(test))

        price  prev_orders  lost  insurance
55818  645.21          126     0     64.521
11488  588.66          143     0     58.866
35753  253.29            0     0     25.329
56240  129.42            0     0     20.000
1116   408.88            0     0     40.888
90324
         price  prev_orders  lost  insurance
98521   658.18           18     0     65.818
11141   118.45            0     0     20.000
110132  274.29          121     0     27.429
101636   82.05            6     0     20.000
67885   123.15            0     0     20.000
22581


In [187]:
x_train = train.iloc[:,[0, 1]]
x_test = test.iloc[:,[0, 1]]
y_train = train.iloc[:, 2]
y_test = test.iloc[:, 2]
i_train = train.iloc[:, 3]
i_test = test.iloc[:, 3]

In [190]:
x_train.isnull().sum()

price          0
prev_orders    0
dtype: int64

In [188]:
y_train.value_counts()

0    85734
1     4590
Name: lost, dtype: int64

As we can see clearly here, the dataset is very clearly unbalanced. So, there is a very large possibility that the model does not "learn" anything, but rather always guesses "0" (do not insure). This is bad for us, though. Hence, we attempt to balance this out by means of calibration. This is achieved through the cross validation calibrated classifier found in scikit learn, and has been implemented below. 

For the model, we are using **logistic regression**, with **auc score** as our evaluation metric.

In [191]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV


clf = LogisticRegression(class_weight='balanced')
calibrated_clf = CalibratedClassifierCV(base_estimator=clf, cv=3, method='isotonic')
calibrated_clf.fit(x_train, y_train.values.ravel())
y_pred = calibrated_clf.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.5011773529363284

Although the AUC value looks bad, it is exactly what is to be expected, since we had randomly initialised the labels.

In [192]:
y_test_pred = pd.DataFrame(y_pred, columns=['prediction'])
y_test_pred.head()

,prediction
0,0.045669
1,0.051855
2,0.052097
3,0.052400
4,0.051855


In [193]:
x_final = x_test
pred = [x[0] for x in y_test_pred.values.tolist()]
x_final['prediction'] = pred
x_final[-10:]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,price,prev_orders,prediction
99384,558.88,0,0.045874
38674,108.08,0,0.052400
47214,125.99,0,0.051855
45402,137.64,0,0.051855
87417,73.59,0,0.052400
104265,478.74,3,0.049133
73774,140.37,2,0.051855
92194,110.92,4,0.052400
46397,93.45,9,0.052400
78626,328.63,1,0.049268


In [194]:
E_x = x_final['price'] * x_final['prediction']
ins = [g(x) for x in x_final['price']]
E_x

98521     30.058293
11141      6.142182
110132    14.289635
101636     4.299395
67885      6.385899
            ...    
104265    23.522021
73774      7.278835
92194      5.812174
46397      4.896751
78626     16.191088
Length: 22581, dtype: float64

In [195]:
x_final['E_x'] = E_x
x_final['ins'] = ins
x_final['lost'] = y_test
x_final

,price,prev_orders,prediction,E_x,ins,lost
98521,658.18,18,0.045669,30.058293,65.818,0
11141,118.45,0,0.051855,6.142182,20.000,0
110132,274.29,121,0.052097,14.289635,27.429,0
101636,82.05,6,0.052400,4.299395,20.000,0
67885,123.15,0,0.051855,6.385899,20.000,0
...,...,...,...,...,...,...
104265,478.74,3,0.049133,23.522021,47.874,0
73774,140.37,2,0.051855,7.278835,20.000,0
92194,110.92,4,0.052400,5.812174,20.000,0
46397,93.45,9,0.052400,4.896751,20.000,0


In [200]:
x_final['price'].sum()

6531261.43

Now, we calculate the expenses we have in different cases.

In [196]:
print("Total cost (insuring everything): ", x_final['ins'].sum())
print("Total cost (insuring everything where insurance cost is less than actual cost): ",
     x_final['ins'][x_final['price']>x_final['ins']].sum()
      +x_final['price'][(x_final['price']<x_final['ins'])&(x_final['lost']==1)].sum())
print("Total cost (insuring according to model): ",
     x_final['ins'][x_final['E_x']>x_final['ins']].sum()
      +x_final['price'][(x_final['E_x']<x_final['ins'])&(x_final['lost']==1)].sum())

Total cost (insuring everything):  748468.0360000001
Total cost (insuring everything where insurance cost is less than actual cost):  743994.6159999999
Total cost (insuring according to model):  291214.25


# Inference
From the above, we can see that by using the model, we can easily save **$457253.78**, which is a pretty decent value.

# Concluding Remarks

There are a couple of things to note, here. For starters, we have used a generated dataset, and results like the one derived above may / may not be actually visible on real world datasets. Also, the model could be improved by adding the following features into consideration:

- Zip code to where the parcel is being delivered
- Medium of shipment (air, freight, cargo might have it's own set of effects on this)
- Company used to facilitate the shipment